In [1]:
from flask import Flask, render_template
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup
import os
from datetime import datetime
import platform
import pdb
import requests

In [2]:
# GET PHASES
def init_browser():
    if platform.system().lower() == 'windows'.lower():
        executable_path = {
            'executable_path': 
            os.path.join(os.getcwd(), 'chromedriver.exe')}
        return Browser('chrome', **executable_path, headless=False)
    else:
        return Browser('chrome')
    
def get_html(browser, url = "https://marvelcinematicuniverse.fandom.com/wiki/Marvel_Cinematic_Universe_Wiki"):
    browser.visit(url)
    html = browser.html
    return html

def get_listings(html):
    
    soup = BeautifulSoup(html, "html.parser")
    listings = soup.find_all('a', class_='wds-dropdown-level-2__toggle')
    return listings
    
def scrape(browser, url):
   
    html = get_html(browser, url)
    listings = get_listings(html)
    return listings


def access_db(dbname, collectionname):

    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    db = client[dbname]
    collection = db[collectionname]
    return db, collection



def main():
        db, collection = access_db('phasesall','mcuphaseall')
        browser = init_browser()
        url='https://marvelcinematicuniverse.fandom.com/wiki/Marvel_Cinematic_Universe_Wiki'
        titles=scrape(browser, url)
        collection.insert_many([{'title':title.text,
                                 'datetime':datetime.now(),'order':order} 
                                    for order, title in enumerate(titles)])
        return

main()



In [3]:
 db, collection = access_db('phasesall','mcuphaseall')

In [4]:
alllist=[data['title'][1:-2] for data in collection.find()]
phaselist = alllist[5:8]
phaselist = [i.replace(" ", "_") for i in phaselist]
phaselist

['Phase_One', 'Phase_Two', 'Phase_Three']

In [2]:
phaselist=['Phase_One','Phase_Two','Phase_Three']

In [3]:
def init_browser():
    if platform.system().lower() == 'windows'.lower():
        executable_path = {
            'executable_path': 
            os.path.join(os.getcwd(), 'chromedriver.exe')}
        return Browser('chrome', **executable_path, headless=False)
    else:
        return Browser('chrome')
    
def get_html(browser, url):
    browser.visit(url)
    html = browser.html
    return html

def get_listings(html):
    soup = BeautifulSoup(html, "html.parser")
    listings = [a.get('href') for a in (i.find('a',href = True) for i in soup.find_all('span', class_='mw-headline')if i) if a]
    return listings

def scrape(browser, url):
    html = get_html(browser, url)
    listings = get_listings(html)
    return listings

def access_db(dbname, collectionname):
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    db = client[dbname]
    collection = db[collectionname]
    return db, collection

def scrape_many(search_terms, collection, browser, base_url):
    for search_term in search_terms:
        url = base_url + search_term
        titles = scrape(browser, url)
        collection.insert_many([
            {'phase':search_term, 'title':title, 
             'datetime':datetime.now(), 'order':order
            } for order, title in enumerate(titles)])
        
def main():
    db, collection = access_db('moviesall','moviesall')
    browser = init_browser()
    base_url = "https://marvelcinematicuniverse.fandom.com/"
    search_terms = phaselist
    scrape_many(search_terms, collection, browser, base_url)
    return 
main()

In [4]:
db, collection = access_db('moviesall','moviesall')
allmoviesphase = [data['phase'] for data in collection.find()]
allmovieslink = [data['title'] for data in collection.find()]

movielinks = dict(zip(allmovieslink, allmoviesphase))
movielinks

{'/wiki/Iron_Man_(film)': 'Phase_One',
 '/wiki/The_Incredible_Hulk': 'Phase_One',
 '/wiki/Iron_Man_2': 'Phase_One',
 '/wiki/Thor_(film)': 'Phase_One',
 '/wiki/Captain_America:_The_First_Avenger': 'Phase_One',
 '/wiki/The_Avengers': 'Phase_One',
 '/wiki/Iron_Man_3': 'Phase_Two',
 '/wiki/Thor:_The_Dark_World': 'Phase_Two',
 '/wiki/Captain_America:_The_Winter_Soldier': 'Phase_Two',
 '/wiki/Guardians_of_the_Galaxy_(film)': 'Phase_Two',
 '/wiki/Avengers:_Age_of_Ultron': 'Phase_Two',
 '/wiki/Ant-Man_(film)': 'Phase_Two',
 '/wiki/Captain_America:_Civil_War': 'Phase_Three',
 '/wiki/Doctor_Strange_(film)': 'Phase_Three',
 '/wiki/Guardians_of_the_Galaxy_Vol._2': 'Phase_Three',
 '/wiki/Spider-Man:_Homecoming': 'Phase_Three',
 '/wiki/Thor:_Ragnarok': 'Phase_Three',
 '/wiki/Black_Panther_(film)': 'Phase_Three',
 '/wiki/Avengers:_Infinity_War': 'Phase_Three',
 '/wiki/Ant-Man_and_the_Wasp': 'Phase_Three',
 '/wiki/Captain_Marvel_(film)': 'Phase_Three',
 '/wiki/Avengers:_Endgame': 'Phase_Three',
 '/wik

In [7]:
base_url = "https://marvelcinematicuniverse.fandom.com/"
titles=[]

for i in movielinks:
    url=base_url+i
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    results= soup.find_all('div', class_="pi-item pi-data pi-item-spacing pi-border-color")

    for result in results:
        content=result.find('h3',class_='pi-data-label pi-secondary-font').text
        titles.append(content)
        
        






In [8]:
final_titles=[]
for title in titles:
    if title=='Running Time' or title=='Box Office':
        final_titles.append(title)
    

    

In [9]:
final_titles

['Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office',
 'Running Time',
 'Box Office']

In [21]:
base_url = "https://marvelcinematicuniverse.fandom.com/"
factors=[]
for j in movielinks:
    r = requests.get(base_url+i)
    soup = BeautifulSoup(r.text, 'html.parser')
    results= soup.find_all('div', class_="pi-item pi-data pi-item-spacing pi-border-color")
    

    for result in results:
        content=result.find('div', class_="pi-data-value pi-font").text
        
        factors.append(content)

print(factors)

['Jon Watts', 'Chris McKennaErik Sommers', "Kevin FeigeAmy PascalLouis D'EspositoVictoria AlonsoThomas M. HammelEric Hauserman CarrollRachel O'ConnorStan LeeAvi AradMatthew Tolmach", 'Michael Giacchino', 'July 2, 2019August 30, 2019 (re-release)', '129 minutes', '$1.127 billion', 'Jon Watts', 'Chris McKennaErik Sommers', "Kevin FeigeAmy PascalLouis D'EspositoVictoria AlonsoThomas M. HammelEric Hauserman CarrollRachel O'ConnorStan LeeAvi AradMatthew Tolmach", 'Michael Giacchino', 'July 2, 2019August 30, 2019 (re-release)', '129 minutes', '$1.127 billion', 'Jon Watts', 'Chris McKennaErik Sommers', "Kevin FeigeAmy PascalLouis D'EspositoVictoria AlonsoThomas M. HammelEric Hauserman CarrollRachel O'ConnorStan LeeAvi AradMatthew Tolmach", 'Michael Giacchino', 'July 2, 2019August 30, 2019 (re-release)', '129 minutes', '$1.127 billion', 'Jon Watts', 'Chris McKennaErik Sommers', "Kevin FeigeAmy PascalLouis D'EspositoVictoria AlonsoThomas M. HammelEric Hauserman CarrollRachel O'ConnorStan LeeAvi

In [22]:
factors

['Jon Watts',
 'Chris McKennaErik Sommers',
 "Kevin FeigeAmy PascalLouis D'EspositoVictoria AlonsoThomas M. HammelEric Hauserman CarrollRachel O'ConnorStan LeeAvi AradMatthew Tolmach",
 'Michael Giacchino',
 'July 2, 2019August 30, 2019 (re-release)',
 '129 minutes',
 '$1.127 billion',
 'Jon Watts',
 'Chris McKennaErik Sommers',
 "Kevin FeigeAmy PascalLouis D'EspositoVictoria AlonsoThomas M. HammelEric Hauserman CarrollRachel O'ConnorStan LeeAvi AradMatthew Tolmach",
 'Michael Giacchino',
 'July 2, 2019August 30, 2019 (re-release)',
 '129 minutes',
 '$1.127 billion',
 'Jon Watts',
 'Chris McKennaErik Sommers',
 "Kevin FeigeAmy PascalLouis D'EspositoVictoria AlonsoThomas M. HammelEric Hauserman CarrollRachel O'ConnorStan LeeAvi AradMatthew Tolmach",
 'Michael Giacchino',
 'July 2, 2019August 30, 2019 (re-release)',
 '129 minutes',
 '$1.127 billion',
 'Jon Watts',
 'Chris McKennaErik Sommers',
 "Kevin FeigeAmy PascalLouis D'EspositoVictoria AlonsoThomas M. HammelEric Hauserman CarrollRac

matching

In [25]:
final_factors=[]
for factor in factors:
    matching=[s for s in factors if 'billion' in s ] 
    final_factors.append(matching)

In [26]:
final_factors

[['$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion'],
 ['$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion'],
 ['$1.127 billion',
  '$1.127 billion',
  '$1.127 billion',
  '$1.127 billion'